In [1]:
import os
os.environ["OPENAI_API_BASE"]= "http://35.221.228.215:8000/v1"
os.environ["OPENAI_API_KEY"]="EMPTY"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain, LLMChain, RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

In [2]:
# qa_prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that exactly answer the question from the context, or if the answer is not inferrable by the context, state so. Write as concise as possible:

# ### Instruction:
#  請依據文章回答：陶朱隱園誰設計的?

# ### Input:
#  陶朱隱園（英語：Tao Zhu Yin Yuan）是位於臺灣臺北市信義計畫區的豪宅大樓，位在松勇路及松高路口，土地面積約為2468坪，由威京集團的中華工程與亞太工商聯合建，建築師為比利時籍的文森·卡利博。該建築樓高93.2公尺，地上樓層21樓，地下樓層4樓，總樓地板面積為12,962坪，於2018年第三季竣工，第一戶的正式交易日期為次年7月3日。

# 建築外觀特殊，屬於少有的旋轉建築，每層樓旋轉4.5度，頂樓設有直升機停機坪，內部則共有七部電梯，其中一部可承載超跑或救護車；其中每戶約三百坪，訴求垂直森林建築，種植超過 2.3 萬顆的喬灌木，裡面甚至還有叢林水瀑布，陽台部分的面積超過五十坪。2018年即完工取得使用執照，直到2019年出現首筆交易，由相關企業公司承耀公司買下7樓戶。過去一兩年實品屋完工後一直沒開放賞屋，市場傳出陶朱隱園價格太高乾脆封盤不賣。

# #### Response:
#  文森·卡利博
 
# -----
# Below is an instruction that describes a task, paired with an input that provides further context. Write a response that exactly answer the question from the context, or if the answer is not inferrable by the context, state so. Write as concise as possible:

# ### Instruction:
#  請依據文章回答，{question}

# ### Input:
#  {context}

# ### Response:
# """ 

In [3]:
# qa_prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that exactly answer the question from the context, or if the answer is not inferrable by the context, state so. Write as concise as possible:
 
# ### Instruction:
#  請依據文章，只擷取文章中的一段文字來回答這個問題：{question}

# ### Input:
#  {context}

# ### Response:
# """ 

In [4]:
qa_prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that exactly answer the question from the context, or if the answer is not inferrable by the context, state so. Write as concise as possible:
 
### Instruction:
 請依據文章，只摘要文章中的一段文字來回答這個問題：{question}

### Input:
 {context}

### Response:
""" 

In [5]:
# qa_prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that exactly answer the question from the context, or if the answer is not inferrable by the context, state so. Write as concise as possible:
 
# ### Instruction:
#  請從文章中擷取一段文字來回答這個問題：{question}

# ### Input:
#  {context}

# ### Response:
# """ 

In [6]:
# qa_prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that exactly answer the question from the context, or if the answer is not inferrable by the context, state so. Write as concise as possible:
 
# ### Instruction:
#  Please extract the answer from the input：{question}

# ### Input:
#  {context}

# ### Response:
# """ 

In [7]:
REDPAJAMA_QA_PROMPT = PromptTemplate(
    template=qa_prompt_template, input_variables=["context", "question"]
)

In [8]:
loader = TextLoader("./fake_esg.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=32, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

Created a chunk of size 42, which is longer than the specified 32
Created a chunk of size 52, which is longer than the specified 32
Created a chunk of size 77, which is longer than the specified 32
Created a chunk of size 88, which is longer than the specified 32
Created a chunk of size 105, which is longer than the specified 32
Created a chunk of size 80, which is longer than the specified 32
Created a chunk of size 86, which is longer than the specified 32
Created a chunk of size 96, which is longer than the specified 32
Created a chunk of size 88, which is longer than the specified 32
Created a chunk of size 92, which is longer than the specified 32
Created a chunk of size 37, which is longer than the specified 32
Created a chunk of size 92, which is longer than the specified 32
Created a chunk of size 82, which is longer than the specified 32
Created a chunk of size 93, which is longer than the specified 32
Created a chunk of size 135, which is longer than the specified 32
Created 

In [9]:
# model_kwargs = {'device': 'cpu'}
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-base",
#     model_kwargs=model_kwargs
)
docsearch = Chroma.from_documents(texts, embeddings)

In [10]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = ChatOpenAI(
        model_name="redpajama-incite-7b-zh-instruct", 
        streaming=True, callbacks=[StreamingStdOutCallbackHandler()],
        temperature=0.0,
        model_kwargs={"top_p":0.0, "frequency_penalty": 1.1})


In [11]:
llm_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    chain_type_kwargs={
        "prompt": REDPAJAMA_QA_PROMPT,
        "verbose": True
    },
)

In [12]:
user_question = "公司有多少員工"

response = llm_chain.run({'query': user_question})
print("response:", response)

print("\n\n")
print("get_relevant_documents:")
for ct, doc in enumerate(llm_chain._get_docs(user_question)):
    print("=================")
    print("ct: ", ct, " len: ", len(doc.page_content))
    print(doc)



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that exactly answer the question from the context, or if the answer is not inferrable by the context, state so. Write as concise as possible:
 
### Instruction:
 請依據文章，只摘要文章中的一段文字來回答這個問題：公司有多少員工

### Input:
 員工數量：
幻境互動有限公司目前有超過500名員工，他們來自各個領域，包括遊戲開發、設計、營銷、技術支持等。我們致力於提供公平、多元和積極的工作環境，鼓勵員工發揮創意和創新精神，並且注重員工的專業成長和福利待遇。

環保議題

員工數量

董事長的話

永續發展報告書

公司概述
1.1 公司名稱和成立日期
1.2 資本額
1.3 總部
1.4 業務範圍

公司幹部
2.1 董事長
2.2 首席執行官
2.3 首席技術官
2.4 首席創意官
2.5 首席財務官

### Response:


根據文章所述，幻境互動有限公司有超過500名員工，其中包含遊戲開發、設計、營銷、技術支持等不同領域的專業人員。他們都受到公平的對待並享有良好的福利待遇。此外，該公司也重視環境保護的議題，在工作場所中實施了相關的措施以保護環境。
> Finished chain.

> Finished chain.
response: 
根據文章所述，幻境互動有限公司有超過500名員工，其中包含遊戲開發、設計、營銷、技術支持等不同領域的專業人員。他們都受到公平的對待並享有良好的福利待遇。此外，該公司也重視環境保護的議題，在工作場所中實施了相關的措施以保護環境。



get_relevant_documents:
ct:  0  len:  106
page_content='員工數量：\n幻境互動有

In [54]:
"""
 請依據文章，只摘要文章中的一段文字來回答這個問題：{question}

Q:
公司的名字是什麼
介紹一下公司
公司如何友善環境？
公司的理念是什麼？
幻境互動有限公司幹部有誰
幻境互動有限公司有多少資本額
公司有哪些社會責任
公司有多少員工
"""

'\n 請依據文章，只摘要文章中的一段文字來回答這個問題：{question}\n\nQ:\n公司的名字是什麼\n介紹一下公司\n公司如何友善環境？\n公司的理念是什麼？\n幻境互動有限公司幹部有誰\n幻境互動有限公司有多少資本額\n公司有哪些社會責任\n公司有多少員工\n'